In [3]:
import random
import numpy as np
import pandas as pd
from scipy import stats
import pycaret
import gower
# import umap
import umap.umap_ as umap
from sklearn.neighbors import DistanceMetric
from pycaret.clustering import *
from category_encoders.backward_difference import BackwardDifferenceEncoder
from transformers import RobertaForMaskedLM, RobertaTokenizer, pipeline
from category_encoders import HelmertEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import gridspec
import matplotlib.pyplot as plt
import plotnine as p9
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# ! не удалять, а добавлять новый признак 

## 1 ML задача

Тебе даны данные HTTP запросов. 

Глобальная задача — найти или разделить вредоносные от хороших. Как мы знаем, вредоносных классов может быть несколько. 

Важнее отделить «мух от котлет». 

Попробуй сделать EDA, понять, а точно ли данные не избыточны и всё, что ты вообще можешь сказать?!

В ходе решения этой части ожидаем, что будут предоставлены артефакты (например jupyter notebook) с экспериментами, которые помогут понять, почему принято решение использовать такой подход к задаче.

## 2 Сервис для inference 
https://heyiamsasha.notion.site/ML-PT-0bc4ce5012604ed397f040a1bdc29858

- Подсказка 1
    
    Классов может быть до 50, но не обязательно 50
    
- Подсказка 2
    
    Кластеризация очень помогает
    
- Подсказка 3
    
    Да, типично что у нас нет числа классов. Но что делать, это жизнь? Мы для базового решения использовали DBSCAN, может быть и ты можешь начать с него?
    
- Подсказка 4
    
    посмотри на поля, точно ли все они нужны?

In [10]:
dataset = pd.read_csv('part_10.csv')
# classified = classified_source.copy()
dataset.sample()

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID
48903,77.232.15.42,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,1550,404,REQUEST_COOKIES,REQUEST_COOKIES.User-cn,Елизавета Сулейманова,S7fXJGQB5cBXmMW1vgzw


In [14]:
dataset.drop(['CLIENT_IP', 'REQUEST_SIZE', 'RESPONSE_CODE', 'EVENT_ID'], axis=1, inplace=True)

In [11]:
num_clusters = 50
random_state = 32

In [17]:
dataset_transformed = pd.read_csv('embeddings.csv') # добавить признаков var_name_eql_value
dataset = pd.concat([dataset_transformed[['REQUEST_SIZE', 'RESPONSE_CODE','bad_resp_code', 'bad_req_size']], dataset], axis=1)

In [18]:
dataset.sample()

,REQUEST_SIZE,RESPONSE_CODE,bad_resp_code,bad_req_size,CLIENT_USERAGENT,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE
35184,1425,404,0,0,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,REQUEST_COOKIES,REQUEST_COOKIES.User-cn,Ангелина Фролова


### Dimensionality Reduction with UMAP

In [19]:
# use Umap to do embedding then cluster on that
def plot_2d_umap(df, intersection=False):

  numerical = df.select_dtypes(exclude='object')

  for c in numerical.columns:
      numerical[c] = (numerical[c] - numerical[c].mean())/numerical[c].std(ddof=0)
      
  ##preprocessing categorical
  categorical = df.select_dtypes(include='object')
  categorical = pd.get_dummies(categorical)

  #Percentage of columns which are categorical is used as weight parameter in embeddings later
  categorical_weight = len(df.select_dtypes(include='object').columns) / df.shape[1]

  #Embedding numerical & categorical
  fit1 = umap.UMAP(random_state=12).fit(numerical)
  fit2 = umap.UMAP(metric='dice', n_neighbors=250).fit(categorical)

  numeric_embedding = fit1.embedding_
  numeric = pd.DataFrame(
                         {'x': numeric_embedding[:,0],
                         'y':  numeric_embedding[:,1],
                        })
  

  categorical_embedding = fit2.embedding_
  categorical = pd.DataFrame(
                         {'x': categorical_embedding[:,0],
                         'y':  categorical_embedding[:,1],
                        })


  # intersection will resemble the numerical embedding more.
  if intersection:
    embedding = fit1 * fit2

  # union will resemble the categorical embedding more.
  else:
    embedding = fit1 + fit2

  umap_embedding = embedding.embedding_


  results = pd.DataFrame(
                        {'x': umap_embedding[:,0],
                         'y':  umap_embedding[:,1],
                        })
  


  numeric = (p9.ggplot(numeric, p9.aes(x='x', y='y'))
              + p9.geom_point(alpha=0.6)
              + p9.ggtitle('Numeric Embedding'))
  
  categorical = (p9.ggplot(categorical, p9.aes(x='x', y='y'))
              + p9.geom_point(alpha=0.6)
              + p9.ggtitle('Categorical Embedding'))
  
  combined = (p9.ggplot(results, p9.aes(x='x', y='y'))
              + p9.geom_point(alpha=0.6)
              + p9.ggtitle('Combined Embedding'))
  
  return numeric, categorical, combined, umap_embedding

In [20]:
UMAP_2D_data = dataset.copy()

In [21]:
%%time
numeric, categorical, combined, embedding_ =  plot_2d_umap(UMAP_2D_data, intersection=True)

UMAP_2D_data['x'] = embedding_[:,0]
UMAP_2D_data['y'] = embedding_[:,1]

# UMAP_2D_data['target'] = labels


UMAP_clusterer = KMeans(n_clusters=num_clusters, init='k-means++',
                        verbose=0, random_state=random_state,
                        algorithm='auto').fit(embedding_)

UMAP_2D_data["preds"] = UMAP_clusterer.labels_
UMAP_2D_data["preds"] = UMAP_2D_data["preds"].apply(str)

predictions['UMAP_2D_preds'] = UMAP_clusterer.labels_

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


NameError: name 'KMeans' is not defined

In [22]:
p1_umap = (p9.ggplot(UMAP_2D_data, p9.aes(x='x', y='y', fill='target', color='target'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data UMAP Ground Truth')
      + p9.theme_bw())

p2_umap = (p9.ggplot(UMAP_2D_data, p9.aes(x='x', y='y', fill='preds', color='preds'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data UMAP Kmeans')
      + p9.theme_bw())

p1_umap

PlotnineError: "Could not evaluate the 'fill' mapping: 'target' (original error: name 'target' is not defined)"

In [23]:
p2_umap

PlotnineError: "Could not evaluate the 'fill' mapping: 'preds' (original error: name 'preds' is not defined)"

### Gower Distance

In [24]:
def calculate_gower_distance(df):
  
  '''
  Takes a dataframe as an input and returns the gower distance
  matrix.
  
  '''

  variable_distances = []
  
  for col in range(df.shape[1]):
    
    feature = df.iloc[:,[col]]
    if feature.dtypes.values == object:
      
      feature_dist = DistanceMetric.get_metric('dice').pairwise(pd.get_dummies(feature, drop_first=True))
        
    else:
      
      feature_dist = DistanceMetric.get_metric('manhattan').pairwise(feature) / max(np.ptp(feature.values),1)


      variable_distances.append(feature_dist)
      

  return np.array(variable_distances).mean(0)

In [ ]:
%%time 
gower_data = dataset.copy()
gower_distance_matrix = calculate_gower_distance(gower_data)

In [ ]:
umap_embeddings = umap.UMAP(random_state=42, n_components=2).fit_transform(gower_distance_matrix)

gower_data['x'] = umap_embeddings[:,0]
gower_data['y'] = umap_embeddings[:,1]

# gower_data['target'] = labels


gower_clusterer = KMedoids(n_clusters=num_clusters, metric='precomputed',
                           method='pam', init='build', 
                           max_iter=300, random_state=random_state).fit(gower_distance_matrix)


gower_data["preds"] = gower_clusterer.labels_
gower_data["preds"] = gower_data["preds"].apply(str)

predictions['gower_preds'] = gower_clusterer.labels_

In [ ]:
p1_gower_d = (p9.ggplot(gower_data, p9.aes(x='x', y='y', fill='target', color='target'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data Gower Matrix UMAP 2D Ground Truth')
      + p9.theme_bw())

p2_gower_d = (p9.ggplot(gower_data, p9.aes(x='x', y='y', fill='preds', color='preds'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data Gower Matrix UMAP 2D KMedoids')
      + p9.theme_bw())

p1_gower_d

In [ ]:
p2_gower_d

### FAMD

In [ ]:
def calculate_zscore(df, columns):
  '''
  scales columns in dataframe using z-score
  '''
  df = df.copy()
  for col in columns:
      df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0)

  return df



def one_hot_encode(df, columns):
  '''
  one hot encodes list of columns and
  concatenates them to the original df
  '''

  concat_df = pd.concat([pd.get_dummies(df[col], drop_first=True, prefix=col) for col in columns], axis=1)
  one_hot_cols = concat_df.columns

  return concat_df, one_hot_cols



def normalize_column_modality(df, columns):
  '''
  divides each column by the probability μₘ of the modality 
  (number of ones in the column divided by N) only for one hot columns
  '''

  length = len(df)
  for col in columns:
    
    weight = math.sqrt(sum(df[col])/length)
    df[col] = df[col]/weight

  return df



def center_columns(df, columns):
  '''
  center columns by subtracting the mean value
  '''
  for col in columns:
      df[col] = (df[col] - df[col].mean())

  return df



def FAMD_(df, n_components=2):
  '''
  Factorial Analysis of Mixed Data (FAMD), 
  which generalizes the Principal Component Analysis (PCA) 
  algorithm to datasets containing numerical and categorical variables

  a) For the numerical variables
    - Standard scale (= get the z-score)
  
  b) For the categorical variables:
    - Get the one-hot encoded columns
    - Divide each column by the square root of its probability sqrt(μₘ)
    - Center the columns

  c) Apply a PCA algorithm over the table obtained!

  '''
  
  variable_distances = []

  numeric_cols = data.select_dtypes(include=np.number)
  cat_cols = data.select_dtypes(include='object')
  
  # numeric process
  normalized_df = calculate_zscore(df, numeric_cols)
  normalized_df = normalized_df[numeric_cols.columns]

  # categorical process
  cat_one_hot_df, one_hot_cols = one_hot_encode(df, cat_cols)
  cat_one_hot_norm_df = normalize_column_modality(cat_one_hot_df, one_hot_cols)
  cat_one_hot_norm_center_df = center_columns(cat_one_hot_norm_df, one_hot_cols)

  # Merge DataFrames
  processed_df = pd.concat([normalized_df, cat_one_hot_norm_center_df], axis=1)

  # Perform (PCA)
  pca = PCA(n_components=n_components)
  principalComponents = pca.fit_transform(processed_df)

  return principalComponents

In [ ]:
FAMD_data = dataset.copy()
FAMD_components = FAMD_(FAMD_data)

FAMD_data['x'] = FAMD_components[:,0]
FAMD_data['y'] = FAMD_components[:,1]

FAMD_data['target'] = labels


FAMD_clusterer = KMeans(n_clusters=num_clusters, init='k-means++',
                        verbose=0, random_state=random_state,
                        algorithm='auto').fit(FAMD_components)


FAMD_data["preds"] = FAMD_clusterer.labels_
FAMD_data["preds"] = FAMD_data["preds"].apply(str)

predictions['FAMD_preds'] = FAMD_clusterer.labels_

In [ ]:
p1_famd = (p9.ggplot(FAMD_data, p9.aes(x='x', y='y', fill='target', color='target'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data FAMD Ground Truth')
      + p9.theme_bw())

p2_famd = (p9.ggplot(FAMD_data, p9.aes(x='x', y='y', fill='preds', color='preds'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data FAMD Kmeans')
      + p9.theme_bw())

p1_famd

In [ ]:
p2_famd

In [ ]:
# пропущенные значения
colours = ['#000099', '#ffff00'] 
sns.set(rc={'figure.figsize':(7,5)})
sns.set(font_scale=0.8)
sns.heatmap(dataset.isnull(), cmap=sns.color_palette(colours))

In [5]:
dataset = dataset[~dataset.isnull().all(1)]

for col in dataset.columns:
    pct_missing = np.mean(dataset[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

CLIENT_IP - 0%
CLIENT_USERAGENT - 1%
REQUEST_SIZE - 0%
RESPONSE_CODE - 0%
MATCHED_VARIABLE_SRC - 0%
MATCHED_VARIABLE_NAME - 8%
MATCHED_VARIABLE_VALUE - 0%
EVENT_ID - 0%


In [6]:
# выбросы
# https://proglib.io/p/moem-dataset-rukovodstvo-po-ochistke-dannyh-v-python-2020-03-27

In [7]:
# TODO: удалить строки - полные дубликаты 

### ДУБЛИКАТЫ  -- ОТСАВИТЬ!!

In [8]:
dataset[dataset.duplicated(keep=False)]

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID
4144,f~|°ØÀ¶ g8%ú·P:V­,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11506,¿b_,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11804,f~|°ØÀ¶ g8%ú·P:V­,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18615,f~|°ØÀ¶ g8%ú·P:V­,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20342,f~|°ØÀ¶ g8%ú·P:V­,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25760,f~|°ØÀ¶ g8%ú·P:V­,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31594,¢×˾û8äyªû¬Aü,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38144,¿b_,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43187,¢×˾û8äyªû¬Aü,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dataset = dataset.drop(index=dataset[dataset.duplicated(keep=False)].index)

In [10]:
# что с этим делать?
key = ['CLIENT_USERAGENT',	'RESPONSE_CODE',	'MATCHED_VARIABLE_SRC',	'MATCHED_VARIABLE_NAME',
       'MATCHED_VARIABLE_VALUE',]
dataset[key][dataset[key].duplicated(keep=False)]

,CLIENT_USERAGENT,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE
27,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,200,REQUEST_GET_ARGS,REQUEST_GET_ARGS.q,"{\""tags\"":[{\""sizes\"":[{\""width\"":728,\""height..."
1922,NaN,NaN,NaN,NaN,NaN
2552,curl/7.22.0 (i686-pc-linux-gnu) libcurl/7.22.0...,200,REQUEST_POST_ARGS,©æ(4È,NaN
3231,NaN,NaN,NaN,NaN,NaN
3371,curl/7.22.0 (i686-pc-linux-gnu) libcurl/7.22.0...,200,REQUEST_POST_ARGS,©æ(4È,NaN
3671,NaN,NaN,NaN,NaN,NaN
6242,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,200,REQUEST_GET_ARGS,REQUEST_GET_ARGS.q,"{\""tags\"":[{\""sizes\"":[{\""width\"":728,\""height..."
6437,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,200,REQUEST_GET_ARGS,REQUEST_GET_ARGS.q,"{\""tags\"":[{\""sizes\"":[{\""width\"":728,\""height..."
7664,NaN,NaN,NaN,NaN,NaN
8035,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,200,REQUEST_GET_ARGS,REQUEST_GET_ARGS.q,"{\""tags\"":[{\""sizes\"":[{\""width\"":728,\""height..."


In [11]:
print('всего CLIENT_IP ', len(dataset['CLIENT_IP'].value_counts().keys()))

print('уникальных CLIENT_IP ', len(set(dataset['CLIENT_IP'].value_counts().keys())))

print('всего EVENT_ID ', len(dataset['EVENT_ID'].value_counts().keys()))

print('уникальных EVENT_ID ', len(set(dataset['EVENT_ID'].value_counts().keys())))

print('всего в датасете записей ', len(dataset))

всего CLIENT_IP  24937
уникальных CLIENT_IP  24937
всего EVENT_ID  54410
уникальных EVENT_ID  54410
всего в датасете записей  57829


In [12]:
# посмотреть на дублирующие EVENT_ID

In [13]:
# REQUEST_SIZE, RESPONSE_CODE в int?

In [14]:
# TODO: кодировки поменять?

### MATCHED_VARIABLE_NAME, MATCHED_VARIABLE_VALUE

In [15]:
dataset[['MATCHED_VARIABLE_SRC', 'MATCHED_VARIABLE_NAME', 'MATCHED_VARIABLE_VALUE' ]].sample(10)

,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE
36962,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,940810849
40467,REQUEST_COOKIES,REQUEST_COOKIES.LtpaToken2,C/NF58NBP96LGFZhT4QdDGaMlxrUMbdGPPKy1/9IZBGh8H...
8547,REQUEST_HEADERS,REQUEST_HEADERS.User-Agent,Mozilla/5.0 (Linux; Android 5.1.1; Ixion P245 ...
5565,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,DOID:USB\\VID_1058&PID_10A2&REV_1042
45959,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516606964697
33662,REQUEST_GET_ARGS,REQUEST_GET_ARGS.e,wqT_3QL4BvBKeAMAAAMA1gAFAQikrMbTBRCg5rOllIzJvX...
48777,REQUEST_ARGS,NaN,\n--boundaryemae\nContent-Disposition: form-da...
25957,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516460766223
13005,REQUEST_POST_ARGS,REQUEST_POST_ARGS.EMAIL,kyayvayv@list.ru
57221,REQUEST_GET_ARGS,id,"172 union all select null,null,null,null,null,..."


In [16]:
len(dataset['MATCHED_VARIABLE_NAME'].value_counts().keys()), len(dataset['MATCHED_VARIABLE_VALUE'].value_counts().keys())

(3209, 57611)

In [17]:
dataset['MATCHED_VARIABLE_NAME'].value_counts()

REQUEST_GET_ARGS._                                                 11165
REQUEST_COOKIES.LtpaToken2                                          3945
REQUEST_PATH                                                        3821
REQUEST_COOKIES.User-cn                                             2444
action                                                              2378
                                                                   ...  
DETAIL_TEXT                                                            1
XâO½#*«
WÒ:©¹µMÖ\täÛk17qòmʂ3³vޡ¥%f®»                           1
srchPat%255B4%255D                                                     1
/content/goods/item[25]:Comment                                        1
XâO½#*«
WÒ:©¹µMÖ\täÛk17qòmʂ3³vާÏ?æÞÅË
Ëî<èMkq,,ÜÆì½8        1
Name: MATCHED_VARIABLE_NAME, Length: 3209, dtype: int64

### MATCHED_VARIABLE_SRC

In [18]:
dataset['MATCHED_VARIABLE_SRC'].value_counts()

REQUEST_GET_ARGS                    20812
REQUEST_COOKIES                      9303
REQUEST_HEADERS                      4930
REQUEST_PATH                         4011
REQUEST_ARGS                         3729
RESPONSE_HEADERS                     3364
REQUEST_POST_ARGS                    3304
REQUEST_URI                          3143
REQUEST_XML                          1959
REQUEST_METHOD\';\'REQUEST_CONTE      812
REQUEST_URI\';\'REQUEST_ARGS          666
REQUEST_ARGS_KEYS                     639
REQUEST_JSON                          587
REQUEST_METHOD\';\'REQUEST_HEADE      227
CLIENT_USERAGENT                      133
CLIENT_SESSION_ID                      42
REQUEST_QUERY                          35
RESPONSE_BODY                          22
REQUEST_CONTENT_TYPE                    6
REQUEST_FILES                           5
CLIENT_IP                               1
Name: MATCHED_VARIABLE_SRC, dtype: int64

In [19]:
dataset[dataset['MATCHED_VARIABLE_SRC'] == 'REQUEST_FILES'].sample(5)

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID
47482,108.179.199.10,Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US...,8884,301,REQUEST_FILES,file,Neko.php\\x0\\x0\\x0\\x0\\x0\\x0\\x0\\x0\\x0\\...,6jV3NmYBrgKk_RFNxhTc
15659,203.162.127.193,Mozilla 15,1049,404,REQUEST_FILES,module,PK\n\\x0\\x0\\x0\\x0\\x0RanIjðýU1\\x0\\x0\\x...,wUvZGmQB5cBXmMW1VuHM
8543,109.195.49.179,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,159543,200,REQUEST_FILES,reference to,http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd,AVdqyNGwq1Ppo9zF-M6B
15121,37.209.206.246,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,968,301,REQUEST_FILES,file,<?php eval(base64_decode(c3lzdGVtKGJhc2U2NF9kZ...,QTDhNWYBrgKk_RFN4OnO
20569,96.44.128.250,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,612,404,REQUEST_FILES,Filedata,8D9AAEEC4D8E4439299046B8CDB3F782<?php @eval($...,ejHrNWYBrgKk_RFNsDI6


In [20]:
### комбинированные признаки

In [21]:
# признак MATCHED_VARIABLE_NAME	== MATCHED_VARIABLE_VALUE	
dataset['var_name_eql_value'] = np.where(dataset['MATCHED_VARIABLE_NAME'] == dataset['MATCHED_VARIABLE_VALUE'], 1, 0 )
dataset['var_name_eql_value'].value_counts()

0    57252
1      577
Name: var_name_eql_value, dtype: int64

In [63]:
np.unique(dataset[dataset['var_name_eql_value'] == 1]['MATCHED_VARIABLE_SRC'])

array(['REQUEST_ARGS', 'REQUEST_ARGS_KEYS', 'REQUEST_GET_ARGS'],
      dtype=object)

In [64]:
# TODO: вытаскивать из CLIENT_USERAGENT curl, linux, mozilla 

In [65]:
# проверка, что MATCHED_VARIABLE_SRC в MATCHED_VARIABLE_NAME
dataset['variable_src_in_name'] = dataset.apply(lambda x: str(x.MATCHED_VARIABLE_SRC) in str(x.MATCHED_VARIABLE_NAME), axis=1) * 1
dataset['variable_src_in_name'].value_counts()

1    41954
0    15891
Name: variable_src_in_name, dtype: int64

In [66]:
dataset[dataset['variable_src_in_name'] == 0].sample(3)

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID,bad_req_size,bad_resp_cod,bad_resp_code,var_name_eql_value,variable_src_in_name
46213,5.143.231.29,Mozilla/5.0 (compatible; SputnikBot/2.3; +http...,1060,302,REQUEST_URI,NaN,"/about/mass-media-reviews/<iframe src=""http:/i...",AVdYcjMdq1Ppo9zF0ZWS,0,0,0,0,0
7006,141.8.132.25,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,682,200,REQUEST_ARGS_KEYS,%2525252525252525D0%2525252525252525B1%2525252...,%2525252525252525D0%2525252525252525B1%2525252...,AVdlH7fcq1Ppo9zF7Krn,0,0,0,0,0
23857,5.255.253.71,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,1051,200,REQUEST_ARGS_KEYS,/tags/core/%2525252525252525252525252525252525...,/tags/core/%2525252525252525252525252525252525...,AVdbDc20q1Ppo9zF1wy_,0,0,0,0,0


### REQUEST_SIZE

In [124]:
dataset['REQUEST_SIZE'] = pd.to_numeric(dataset['REQUEST_SIZE'], errors='coerce')
dataset.loc[50, 'REQUEST_SIZE']

nan

In [125]:
dataset['bad_req_size'] = np.where(dataset['REQUEST_SIZE'].isna(), 1, 0)

In [126]:
# не удалять, а добавлять новый признак 
dataset[dataset['REQUEST_SIZE'].isna()]

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID,MATCHED_VARIABLE_SRC_median,MATCHED_VARIABLE_SRC_sum,...,MATCHED_VARIABLE_VALUE_max,MATCHED_VARIABLE_VALUE_std,EVENT_ID_median,EVENT_ID_sum,EVENT_ID_mean,EVENT_ID_mode,EVENT_ID_min,EVENT_ID_max,EVENT_ID_std,bad_req_size
50,"> -gOxT""O<Ghèe]Ø È8 G`Qf¾ñ }y ðù!T...",SQL Injection,NaN,2f53b17df6da6bf9aeec27b0d87413c4,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
162,Remote Code Execution,AVdtgf8uq1Ppo9zF_hyd,NaN,NaN,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
1308,"~@.*"")ۅ",SQL Injection,NaN,dd7ff149a6270e8037f6aa280b4ca1bd,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
1894,Remote Code Execution,AVddr1QQq1Ppo9zF3Vuj,NaN,NaN,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
3344,Remote Code Execution,AVddRtA4q1Ppo9zF3E0u,NaN,NaN,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51849,"«è%5g§ê""Ze:^ Z%ØÓ)ù¨sT´Õ",SQL Injection,NaN,d544692b358f118894bd292a297f5635,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
55418,Á5ÎBÒÀìv|¤m*Ýí^¬±yº¿0Gåÿ,Cross-Site Scripting,NaN,be917628f6edcdc86d6bdea6edab6c48,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
55816,>ݢÕ òæ%lߟÓ 3ÿ,SQL Injection,NaN,7ccc451ad8a2981f1e6f7eb6a5813246,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1
55854,Remote Code Execution,AVdcPTKlq1Ppo9zF2cLI,NaN,NaN,nan,nan,nan,nan,755.5,1511,...,1429,673.5,755.5,1511,755.5,[82],82,1429,673.5,1


In [127]:
dataset['REQUEST_SIZE'].fillna(-1, inplace=True)
dataset['REQUEST_SIZE'] = dataset['REQUEST_SIZE'].astype('int')

### RESPONSE_CODE

In [128]:
dataset['RESPONSE_CODE'].value_counts()[:1]

200    25771
Name: RESPONSE_CODE, dtype: int64

In [129]:
dataset['RESPONSE_CODE'].value_counts()[-3:]

82b4c42ac7edc647d27de21ceb791491    1
429                                 1
7ccc451ad8a2981f1e6f7eb6a5813246    1
Name: RESPONSE_CODE, dtype: int64

In [130]:
dataset['RESPONSE_CODE'] = pd.to_numeric(dataset['RESPONSE_CODE'], errors='coerce')
dataset['bad_resp_code'] = np.where(dataset['RESPONSE_CODE'].isna(), 1, 0)
dataset[dataset['RESPONSE_CODE'].isna()]

,CLIENT_IP,CLIENT_USERAGENT,REQUEST_SIZE,RESPONSE_CODE,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID,MATCHED_VARIABLE_SRC_median,MATCHED_VARIABLE_SRC_sum,...,MATCHED_VARIABLE_VALUE_std,EVENT_ID_median,EVENT_ID_sum,EVENT_ID_mean,EVENT_ID_mode,EVENT_ID_min,EVENT_ID_max,EVENT_ID_std,bad_req_size,bad_resp_code
50,"> -gOxT""O<Ghèe]Ø È8 G`Qf¾ñ }y ðù!T...",SQL Injection,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,1,1
162,Remote Code Execution,AVdtgf8uq1Ppo9zF_hyd,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,1,1
1308,"~@.*"")ۅ",SQL Injection,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,1,1
1894,Remote Code Execution,AVddr1QQq1Ppo9zF3Vuj,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,1,1
1922,Ä}jSL 0,nan,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55816,>ݢÕ òæ%lߟÓ 3ÿ,SQL Injection,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,1,1
55854,Remote Code Execution,AVdcPTKlq1Ppo9zF2cLI,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,1,1
55871,ÕBÿ,AVddkIdNq1Ppo9zF3QzL,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,0,1
55896,Remote Code Execution,AVdcvSoAq1Ppo9zF2y_z,-1,NaN,nan,nan,nan,nan,755.5,1511,...,673.5,755.5,1511,755.5,[82],82,1429,673.5,1,1


In [131]:
dataset['RESPONSE_CODE'].fillna(-1, inplace=True)
dataset['RESPONSE_CODE'] = dataset['RESPONSE_CODE'].astype('int')

In [132]:
### UUID?

In [77]:
rndm = 3333
for i in range(rndm,(rndm+10)):
    print(dataset.loc[i, 'MATCHED_VARIABLE_VALUE'])
    print()

1516677639247

http://www.surgedigital.co.uk/renova-toilet-paper-uk.pdf

(/**/sElEcT 1 /**/fRoM(/**/sElEcT count(*),/**/cOnCaT((/**/sElEcT(/**/sElEcT(/**/sElEcT /**/cOnCaT(0x217e21,ifnull(user_password,char(32)),0x217e21) /**/fRoM u20639.phpbb_users /**/lImIt 230,1)) /**/fRoM information_schema./**/tAbLeS /**/lImIt 0,1),floor(rand(0)*2))x /**/fRoM information_schema./**/tAbLeS /**/gRoUp/**/bY x)a)

30450220690345aaaf4886be3a7d67f72acf786b6442b3f7ca52358e27254a75ab3342aa022100a9757e9f881e4baac4540640278a3e16cc4de7e84709a4040cd5e7f9e9c2adb4:562fb376085b477c52a3686e30e84d005131c68fc81b194ee572719ffcc4913c

reporttm;topic=367.68;msg=139489

/mail/mad/vbolis.nsf/($Inbox)/2cf1e5e1af8ca26dc12582360036b7d9/body/27.31B2

verificationcode;vid=register;rand=ff33248bf790331a32947499e6bb2963

Ольга Волчкова

Au6ATH2kDS1NVjHNegzWqrNUQx7KiZoR5YUWotEYWvI68iHdd24MR9M3Nmr88FINcxwWrjmwr9eWlyQMwhUaB2AZanSYUMqiBk5zYDePMPhoqDcCe7OWdIcmJTfyHBa25f54dTXwWTzBEeMeWLZn21qCsgt33Yrv0lH3LAcvnjLN6b8DeQdPIFWL82ljQFjvw

In [78]:
cat_features = list(dataset.loc[:, dataset.dtypes == object].columns)

num_features = list(set(dataset.columns).difference(set(cat_features)))

for col in cat_features:
    dataset[col + '_count'] = dataset[col].map(dataset.groupby(col).size())

In [80]:
# TODO: признак по апишке 
cat_features.remove('CLIENT_IP')

In [81]:
dataset[cat_features]

,CLIENT_USERAGENT,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE,EVENT_ID
0,NaN,REQUEST_URI,NaN,//tmp/20160925122692indo.php.vob,AVdhXFgVq1Ppo9zF5Fxu
1,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:45.0) G...,REQUEST_GET_ARGS,url,http://www.galitsios.gr/?option=com_k2,AVdcJmIIq1Ppo9zF2YIp
2,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,REQUEST_PATH,NaN,"/opinion/page1_3.php\"" and \""\""x\""\""=\""\""x\""",iz7SN2YBrgKk_RFNZW_U
3,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:45.0) G...,REQUEST_GET_ARGS,url,http://thereviewgroup.org/boss-capital/,AVdjekw4q1Ppo9zF6QT2
4,Mozilla/5.0 (Linux; Android 4.4.2; GT-N7100 Bu...,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516288084140,SqQGI2QB5cBXmMW1CDbp
...,...,...,...,...,...
57840,MICROSOFT_DEVICE_METADATA_RETRIEVAL_CLIENT,REQUEST_XML,REQUEST_XML./s:Envelope/s:Body/DeviceMetadataB...,DOID:USBSTOR\\Imation_USB_Flash_Drive_2,tmtVHmQB5cBXmMW1MRdz
57841,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,REQUEST_GET_ARGS,REQUEST_GET_ARGS._,1516440395935,Am-kHmQB5cBXmMW1YAcE
57842,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:57...,REQUEST_COOKIES,REQUEST_COOKIES.User-cn,Николай Харченко,5lKlG2QB5cBXmMW1ghFl
57843,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,REQUEST_POST_ARGS,REQUEST_POST_ARGS.s_Unids,688D71625B88803CC125818E00471A8D;9915FD3520134...,OsTJ_mMBjksgoq1e2QnM


In [83]:
tokenizer = RobertaTokenizer.from_pretrained('sberbank-ai/ruRoberta-large')

def get_token(x):
    """возвращает эмбеддинг текста"""
    return tokenizer(x, add_special_tokens=False)['input_ids']

dataset['random_feature'] = random.sample(range(1, len(dataset) + 1), len(dataset))

In [100]:
rndm = 3333
for i in range(rndm,(rndm+10)):
    print(dataset.loc[i, 'CLIENT_USERAGENT'])
    # print(type(dataset.loc[i, 'CLIENT_USERAGENT']))

Mozilla/5.0 (Linux; Android 7.0; SM-A710F Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.111 Mobile Safari/537.36
Mozilla/5.0 (compatible; AhrefsBot/5.1; +http://ahrefs.com/robot/)
Mozilla/5.0 (Windows; U; Windows NT 5.1; pt-PT; rv:1.9.1.2) Gecko/20090729 Firefox/3.5.2 (.NET CLR 3.5.30729)
Google Update/1.3.33.7;winhttp;cup-ecdsa
Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)
Mozilla/5.0 (Linux; Android 7.0; LG-M200 Build/NRD90U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.137 Mobile Safari/537.36
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36
Mozilla/5.0 (iPhone; CPU iPhone OS 11_2_2 like Mac OS X) AppleWebKit/604.4.7 (KHTML, like Gecko) Version/11.0 Mobile/15C202 Safari/604.1
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.12

In [113]:
# TODO: как-то обрабаывать List[str] и List[List[str]] ?

In [118]:
for col in cat_features:
    dataset[col] = dataset[col].astype('str')

In [119]:
# CLIENT_USERAGENT и MATCHED_VARIABLE_VALUE долго обрабатываются 
text_cols = cat_features[1:]

In [136]:
%%time
for col in text_cols:
    if col != text_cols[-1]:
        print(col, ' -->')
    dataset[col + '_median'] = dataset[col].apply(lambda x: np.median(get_token(x)))
    dataset[col + '_sum'] = dataset[col].apply(lambda x: np.sum(get_token(x)))
    dataset[col + '_mean'] = dataset[col].apply(lambda x: np.mean(get_token(x)))
    dataset[col + '_mode'] = dataset[col].apply(lambda x: stats.mode(get_token(x)).mode[0])
    dataset[col + '_min'] = dataset[col].apply(lambda x: np.min(get_token(x)))
    dataset[col + '_max'] = dataset[col].apply(lambda x: np.max(get_token(x)))
    dataset[col + '_std'] = dataset[col].apply(lambda x: np.std(get_token(x)))

CLIENT_USERAGENT  -->
MATCHED_VARIABLE_SRC  -->
MATCHED_VARIABLE_NAME  -->
MATCHED_VARIABLE_VALUE  -->
EVENT_ID  -->
CPU times: user 49.3 s, sys: 58.4 ms, total: 49.4 s
Wall time: 49.5 s


In [141]:
num_features = list(set(dataset.columns).difference(set(cat_features)))
num_features.append('CLIENT_IP')
len(num_features)

41

In [143]:
dataset[num_features].to_csv('embeddings.csv', index=False)

In [5]:
dataset_transformed = pd.read_csv('embeddings.csv')

In [33]:
# пока без теста
# X_train, X_test, y_train, y_test = train_test_split(classified, y, test_size=0.2, random_state=42)

In [34]:
# %%time
# bd = BackwardDifferenceEncoder(drop_invariant=True)
# dataset_bd = bd.fit_transform(dataset[cat_features])
# dataset_f = pd.merge(dataset_bd, dataset[num_features], left_index=True, right_index=True)

In [ ]:
# %%time
# he = HelmertEncoder(drop_invariant=True)
# dataset_he = he.fit_transform(dataset[cat_features])
# dataset_f = pd.merge(dataset_he, dataset[num_features], left_index=True, right_index=True)

In [ ]:
# https://www.kaggle.com/code/karnikakapoor/customer-segmentation-clustering

In [ ]:
# https://github.com/art-glazunov/Non-standard-categorical-encodings/blob/main/group_counts_and_SVD.ipynb

In [8]:
# брать только часть датасета 
df_sample = dataset_transformed.sample(frac=0.25)
dataset_transformed.shape, df_sample.shape

((57845, 41), (14461, 41))

### Goverce distance 

In [ ]:
# https://medium.com/analytics-vidhya/the-ultimate-guide-for-clustering-mixed-data-1eefa0b4743b

In [ ]:
## очень долго считается

%%time
dist_matrix = gower.gower_matrix(df_sample)

In [ ]:
from sklearn_extra.cluster import KMedoids

clusters = (KMedoids(n_clusters=num_clusters, 
                           metric='precomputed',
                           method='pam', init='build', 
                           max_iter=300, 
                           random_state=random_state)
            .fit(dist_matrix)
            .labels_)

### pycaret 

In [5]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (54953, 8)
Unseen Data For Predictions: (2892, 8)


In [7]:
exp_clu101 = setup(data, normalize = True, 
                   # ignore_features = ['MouseID'],
                   session_id = 123)

KeyboardInterrupt: 

In [ ]:
# 'dbscan'
kmeans = create_model('kmeans')
print(kmeans)

In [ ]:
kmodes = create_model('kmodes', num_clusters = 6)
print(kmodes)

In [ ]:
models()

In [ ]:
kmean_results = assign_model(kmeans)
kmean_results.head()

In [ ]:
plot_model(kmeans)

In [ ]:
plot_model(kmeans, plot = 'elbow')

In [ ]:
plot_model(kmeans, plot = 'silhouette')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'class')

In [ ]:
unseen_predictions = predict_model(kmeans, data=data_unseen)
unseen_predictions.head()

In [ ]:
save_model(kmeans,’Final KMeans Model 25Nov2020')

In [ ]:
# TODO: посмотреть на метрики - насколько хорошо обучилась модель 